## Install the Data

In [1]:
%pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.3 MB/s 
     |████████████████████████████████| 120 kB 60.1 MB/s 
     |████████████████████████████████| 115 kB 56.1 MB/s 
     |████████████████████████████████| 212 kB 49.5 MB/s 
     |████████████████████████████████| 127 kB 56.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import datasets

In [3]:
all_ds = datasets.list_datasets()
len(all_ds)

8781

In [5]:
dataset = datasets.load_dataset('oscar', 'unshuffled_deduplicated_az')

Generating train split:   0%|          | 0/626796 [00:00<?, ? examples/s]

Dataset oscar downloaded and prepared to /root/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_az/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 626796
    })
})

In [7]:
dataset['train'][0]

{'id': 0,
 'text': 'AZTV-Artıq 7 ildir ki, Abşeron rayonu dotasiya almadan bütün xərclərini yerli daxilolmalar hesabına maliyyələşdirir.\nDünən, 10:49 | Azərbaycanda Konstitusiya GünüdürDünən, 10:46 | Yapon tədqiqatçılar Parkinson xəstəliyinin müalicəsi istiqamətində mühüm uğur qazanıblarDünən, 10:44 | Azərbaycan İstanbul Beynəlxalq Kitab Sərgisində təmsil olunur10-11-2018, 19:56 | İradə Gülməmmədova şahmat üzrə üçqat Avropa çempionu ilə görüşdü- FOTO10-11-2018, 11:18 | Süni intellektli ilk bankomat istifadəyə verilib10-11-2018, 11:16 | Çikaqoda Dövlət Bayrağı Günü münasibətilə tədbir keçirilib10-11-2018, 11:12 | Azərbaycan ilə Türkiyə arasında QHT-lərlə bağlı memorandum imzalanıb9-11-2018, 19:05 | Abşeronun yeniyetmə cüdoçuları zona mərhələsinin qalibi adını qazanıblar9-11-2018, 16:44 | Ceyranbatan 1 nömrəli məktəbdə Bayraq Günü tədbiri-Video9-11-2018, 15:25 | Xırdalanın 23 saylı bağçasında Dövlət Bayrağı Günü qeyd olunub (Foto+Video)9-11-2018, 15:09 | Xırdalan şəhər 3 saylı uşaq bağç

In [8]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we will have ~3808 leftover samples, we save those now too
with open(f'text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))


  0%|          | 0/626796 [00:00<?, ?it/s]

## Build a Custom Transformer Tokenizer

In [9]:
from pathlib import Path
import os

In [10]:
paths = [str(x) for x in Path('').glob('**/*.txt')]
paths

['text_39.txt',
 'text_22.txt',
 'text_91.txt',
 'text_53.txt',
 'text_75.txt',
 'text_112.txt',
 'text_11.txt',
 'text_55.txt',
 'text_98.txt',
 'text_93.txt',
 'text_14.txt',
 'text_9.txt',
 'text_107.txt',
 'text_68.txt',
 'text_18.txt',
 'text_117.txt',
 'text_78.txt',
 'text_13.txt',
 'text_47.txt',
 'text_7.txt',
 'text_16.txt',
 'text_57.txt',
 'text_85.txt',
 'text_20.txt',
 'text_31.txt',
 'text_121.txt',
 'text_48.txt',
 'text_104.txt',
 'text_23.txt',
 'text_114.txt',
 'text_51.txt',
 'text_108.txt',
 'text_92.txt',
 'text_41.txt',
 'text_29.txt',
 'text_102.txt',
 'text_6.txt',
 'text_5.txt',
 'text_111.txt',
 'text_122.txt',
 'text_70.txt',
 'text_45.txt',
 'text_46.txt',
 'text_76.txt',
 'text_113.txt',
 'text_60.txt',
 'text_19.txt',
 'text_101.txt',
 'text_0.txt',
 'text_67.txt',
 'text_28.txt',
 'text_61.txt',
 'text_72.txt',
 'text_77.txt',
 'text_17.txt',
 'text_54.txt',
 'text_8.txt',
 'text_44.txt',
 'text_74.txt',
 'text_56.txt',
 'text_4.txt',
 'text_58.txt',
 't

In [11]:
%pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 4.1 MB/s 


In [12]:
from tokenizers import ByteLevelBPETokenizer

In [13]:
tokenizer = ByteLevelBPETokenizer()

In [ ]:
tokenizer.train(files=paths, vocab_size=30_522, min_frequency=2,
                    special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [ ]:
os.mkdir('bert_dayi')

tokenizer.save_model('bert_dayi')

#### Using the tokenizer

In [ ]:
%pip install transformers

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained('bert_dayi') # load the tokenizer

In [ ]:
lorem_ipsum = (
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor "
    "incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud "
    "exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute "
    "irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla "
    "pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia "
    "deserunt mollit anim id est laborum."
)

In [ ]:
tokenizer(lorem_ipsum, max_length=512, padding='max_length', truncation=True)

## Building MLM Training Input Pipeline

In [ ]:
with open('text_0.txt', 'r', encoding='utf-8') as fp:
    lines = fp.read().split('\n')

In [ ]:
batch = tokenizer(lines, max_length=512, padding='max_length', truncation=True)
len(batch)

2

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
import torch

def mlm(tensor):
    rand = torch.rand(tensor.shape)
    mask_arr = rand < 0.15 * (tensor > 2)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero())
        tensor[i, selection] = 4
    return tensor

In [ ]:
from pathlib import Path

paths = [str(x) for x in Path('').glob('*.txt')]
paths[:5]

['text_0.txt', 'text_1.txt', 'text_2.txt', 'text_3.txt']

In [ ]:
from tqdm.auto import tqdm

input_ids = []
mask = []
labels = []

for path in tqdm(paths):
    with open(path, 'r', encoding='utf-8') as fp:
        lines = fp.read().split('\n')
    sample = tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
    # input_ids.append(batch['input_ids'])
    # mask.append(batch['attention_mask'])
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids.detach().clone()))

100%|██████████| 4/4 [00:15<00:00,  3.80s/it]


In [ ]:
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)

In [ ]:
input_ids[0][:10]

tensor([    0,    44,   837,   338,  7598, 21561,    30, 21561,   610, 14601])

In [ ]:
encodings = {
    'input_ids': input_ids,
    'attention_mask': mask,
    'labels': labels
}

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encoding):
        self.encoding = encoding
    def __len__(self):
        return self.encoding['input_ids'].shape[0]
    def __getitem__(self, idx):
        return {key: tensor[idx] for key, tensor in self.encoding.items()}

In [ ]:
dataset = Dataset(encodings)

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

## Training and Testing Latin BERT

In [ ]:
from transformers import RobertaConfig
config =  RobertaConfig(
    vocab_size=tokenizer.vocab_size,
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaForMaskedLM

In [ ]:
model = RobertaForMaskedLM(config)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [ ]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor